# Object Tracking with Ultranalytics

In [ ]:
# !pip install ultralytics

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo track source='/content/drive/MyDrive/datasets_and_files/car_flow.mp4' save=True

In [ ]:
video_path = '/content/drive/MyDrive/datasets_and_files/car_flow.mp4'

output_path = "object_tracking_yolov8.mp4"

In [ ]:
model = YOLO("/content/drive/MyDrive/yolov8n.pt")

names = model.model.names

track_history = defaultdict(lambda: [])

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):

                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))

                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break


result.release()
cap.release()
cv2.destroyAllWindows()


from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Object Counting

In [ ]:
# !pip install ultralytics

In [63]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2
from ultralytics import solutions


In [ ]:
video_path = '/content/drive/MyDrive/datasets_and_files/people.m4v'

output_path = "people_counting_yolov8.mp4"

In [61]:
region_points = [
    (359, 138),
    (359, 388),
    (409, 388),
    (409, 138)
]

In [ ]:
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

counter = solutions.ObjectCounter(
    show=True,
    region=region_points,
    model="/content/drive/MyDrive/yolov8n.pt",
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.count(im0)
    video_writer.write(im0)


cap.release()
video_writer.release()
cv2.destroyAllWindows()

from google.colab import files
files.download(output_path)

Ultralytics Solutions: ✅ {'region': [(359, 138), (359, 388), (409, 388), (409, 138)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'view_img': True, 'reg_pts': [(359, 138), (359, 388), (409, 388), (409, 138)], 'classes_names': {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil, os
from glob import glob

In [ ]:
output_path = '/content/people_counting_yolov8.mp4'

In [ ]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/outputs/'

In [ ]:
os.makedirs(drive_path, exist_ok=True)

In [ ]:
shutil.copy(output_path, drive_path)

'/content/drive/MyDrive/Colab Notebooks/outputs/people_counting_yolov8.mp4'

In [ ]:
os.listdir(drive_path)

['people_counting_yolov8.mp4']

# Saving Outputs from Colab to Drive

In [ ]:
import shutil, os
from glob import glob

output_path = '/content/people_counting_yolov8.mp4'  
drive_path = '/content/drive/MyDrive/Colab Notebooks/outputs/'  

In [ ]:
os.makedirs(drive_path, exist_ok=True)

In [ ]:
shutil.copy(output_path, drive_path)

'/content/drive/MyDrive/Colab Notebooks/outputs/people_counting_yolov8.mp4'

In [ ]:
os.listdir(drive_path)

['people_counting_yolov8.mp4']